In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
df_teste = pd.read_parquet('/content/drive/MyDrive/df_teste_norm')
df_treino = pd.read_parquet('/content/drive/MyDrive/df_treino_norm')
df_treino = df_treino.dropna()
df_teste = df_teste.dropna()
df_treino2 = df_treino[['TEMP_NORM','BATIMETRIA_NORM']]
df_teste2 = df_teste[['TEMP_NORM','BATIMETRIA_NORM']]
lista = list()
for x,i in df_treino2.values:
  lista.append([x,i])
sal_treino = np.array(df_treino.SALINIDADE)
pr_treino = np.array(lista)
lista = list()
for x,i in df_teste2.values:
  lista.append([x,i])
sal_teste = np.array(df_teste.SALINIDADE)
pr_teste = np.array(lista)
pr_teste2 = pr_teste.reshape(pr_teste.shape[0],pr_teste.shape[1],1)
pr_treino2 = pr_treino.reshape(pr_treino.shape[0],pr_treino.shape[1],1)

In [ ]:
def redeDensa():
  input = tf.keras.layers.Input(2)
  dense = tf.keras.layers.Dense(30,activation='relu')(input)
  dense = tf.keras.layers.Dense(30,activation='relu')(dense)
  dense = tf.keras.layers.Dropout(0.2)(dense)
  outp = tf.keras.layers.Dense(1)(dense)
  modelo = tf.keras.Model(inputs=input, outputs=outp)
  adan = tf.keras.optimizers.Adam(learning_rate=0.001)
  modelo.compile(optimizer=adan, loss='mse', metrics=['mae','mse'])
  return modelo

def redeLSTM():
  input = tf.keras.layers.Input(shape=(2,1), name='Bat_Temp')
  lstm = tf.keras.layers.LSTM(units=30,activation='relu')(input)
  output = tf.keras.layers.Dense(1,name='salinidade')(lstm)
  modelo = tf.keras.Model(inputs=input, outputs=output)
  adan = tf.keras.optimizers.Adam(learning_rate=0.007)
  modelo.compile(optimizer=adan, loss='mse', metrics=['mae','mse'])
  return modelo

def redeGRU():
  input = tf.keras.layers.Input(shape=(2,1), name='Bat_Temp')
  GRU = tf.keras.layers.GRU(units=85,activation='relu')(input)
  output = tf.keras.layers.Dense(1,name='salinidade')(GRU)
  modelo = tf.keras.Model(inputs=input, outputs=output)
  adan = tf.keras.optimizers.Adam(learning_rate=0.001)
  modelo.compile(optimizer=adan, loss='mse', metrics=['mae','mse'])
  return modelo

def redeBi_LSTM():
  input = tf.keras.layers.Input(shape=(2,1), name='Bat_Temp')
  lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=60,activation='relu',return_sequences=True))(input)
  lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=60,activation='relu'))(lstm)
  output = tf.keras.layers.Dense(1,name='salinidade')(lstm)
  modelo = tf.keras.Model(inputs=input, outputs=output)
  adan = tf.keras.optimizers.Adam(learning_rate=0.005)
  modelo.compile(optimizer=adan, loss='mse', metrics=['mae','mse'])
  return modelo

def redeBi_GRU():
  input = tf.keras.layers.Input(shape=(2,1), name='Bat_Temp')
  GRU = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=75,activation='relu',return_sequences=True))(input)
  GRU = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=75,activation='relu'))(GRU)
  output = tf.keras.layers.Dense(1,name='salinidade')(GRU)
  modelo = tf.keras.Model(inputs=input, outputs=output)
  adan = tf.keras.optimizers.Adam(learning_rate=0.001)
  modelo.compile(optimizer=adan, loss='mse', metrics=['mae','mse'])
  return modelo 

In [ ]:
denseR = redeDensa()
lstmR = redeLSTM()
gruR = redeGRU()
bilstmR = redeBi_LSTM()
bigruR = redeBi_GRU()

In [ ]:
denseR.fit(pr_treino,sal_treino,epochs=39,batch_size=120)
x_dense,y_dense,z_dense = denseR.evaluate(pr_teste,sal_teste)
print('MSE: {} \n MAE: {}'.format(x_dense,y_dense))

In [ ]:
lstmR.fit(pr_treino2,sal_treino,epochs=85,batch_size=120)
x_lstm,y_lstm,z_lstm = lstmR.evaluate(pr_teste2,sal_teste)
print('MSE: {} \n MAE: {}'.format(x_lstm,y_lstm))

In [ ]:
gruR.fit(pr_treino2,sal_treino,epochs=99,batch_size=120)
x_gru,y_gru,z_gru = gruR.evaluate(pr_teste2,sal_teste)
print('MSE: {} \n MAE: {}'.format(x_gru,y_gru))

In [ ]:
bilstmR.fit(pr_treino2,sal_treino,epochs=98,batch_size=120)
x_bilstm,y_bilstm,z_bilstm = bilstmR.evaluate(pr_teste2,sal_teste)
print('MSE: {} \n MAE: {}'.format(x_bilstm,y_bilstm))

In [ ]:
bigruR.fit(pr_treino2,sal_treino,epochs=100,batch_size=120)
x_bigru,y_bigru,z_bigru = bigruR.evaluate(pr_teste2,sal_teste)
print('MSE: {} \n MAE: {}'.format(x_bigru,y_bigru))

----------Dense------------


MAE: 0.9072026014328003

MSE: 4.460409641265869 


---------LSTM--------------


MAE: 0.8655350208282471

MSE: 4.325561046600342


---------GRU---------------


MAE: 0.8504917025566101

MSE: 4.337936878204346 


----------BI-LSTM---------


MAE: 0.8729144930839539

MSE: 4.260308265686035 


---------BI-GRU------------


MAE: 0.8751853108406067

MSE: 4.290380477905273 
 